# OpenAI Robotics Interaction Demo

In [1]:
import os
import openai
import json

# Load your API key
key_file = open("../openai_key.txt","r")
openai.api_key = key_file.read()
openai_engine = "code-davinci-002"

### Load the robot interface
This is the preamble that will be used to set the context for the call to the codex model. 

These methods are unimplemented, mainly to keep the model input short and to-the-point. Later, this will be swapped for the implemented version to run on the robot.

In [2]:
f = open("robot_interface.py","r")
template = f.read()

print(template)

# Python 3
# Interface for a robot arm

class Color(enum.Enum):
    Blue =   "blue"   #blue
    Green =  "green"  #green
    Yellow = "yellow" #yellow
    Red =    "red"    #red
    Orange = "orange" #orange
    Black =  "black"  #black
    
class SceneObjectType(enum.Enum):
    Block =  "block"  #block
    Bin =    "bin"    #bin
    Ball =   "ball"   #ball
    
class Pose:

class SceneObject:
    def get_color(self) -> Color:
        """Gets the color of the scene object"""
    
    def get_object_type(self) -> SceneObjectType:
        """Gets the object type of the scene object"""
    
    def get_pose(self) -> Pose:
        """Gets the pose of the scene object"""
        
class RobotController:
    def __init__(self):
        """RobotController initializer"""
        
    def rotate_left(self, radians):
        """Rotate the arm left by the amount of radians specified"""
        
    def rotate_right(self, radians):
        """Rotate the arm right by the amount of radians specified"

### Create your prompt

In [3]:
good_examples =  """

# Pick up the blue block, then rotate 30 degrees to the left, and drop it.

# Initialize the robot
robot = RobotController()

# Get the list of objects in the scene
scene_objects = robot.get_scene_objects()

# Find the blue block
blue_block = [obj for obj in scene_objects if obj.get_color() == Color.Blue and obj.get_object_type() == SceneObjectType.Block][0]

# Open the gripper
robot.release()

# Go to the blue block
robot.go_to_pose(blue_block.get_pose())

# Tilt down to the grasp position
robot.tilt_down(0.075)

# Grasp the blue block
robot.grasp()

# Lift up the block
robot.tilt_up(0.5)

# Rotate left 30 degrees
robot.rotate_left(math.pi / 6)

# Place the block down
robot.tilt_down(0.5)

# Drop the blue block
robot.release()

###

# Move the red block 0.17 radians forward

# Initialize the robot
robot = RobotController()

# Get the list of objects in the scene
scene_objects = robot.get_scene_objects()

# Find the red block
red_block = [obj for obj in scene_objects if obj.get_color() == Color.Red and obj.get_object_type() == SceneObjectType.Block][0]

# Open the gripper
robot.release()

# Go to the red block
robot.go_to_pose(red_block.get_pose())

# Tilt down to the grasp position
robot.tilt_down(0.075)

# Grasp the red block
robot.grasp()

# Lift up the block
robot.tilt_up(0.5)

# Move forward 0.17 radians
robot.move_forward(0.17)

# Place the block down
robot.tilt_down(0.5)

# Drop the red block
robot.release()

###

# Put the red blocks in the blue bin

# Initialize the robot
robot = RobotController()

# Get the list of objects in the scene
scene_objects = robot.get_scene_objects()

# Find the blue bin
blue_bin = [obj for obj in scene_objects if obj.get_color() == Color.Blue and obj.get_object_type() == SceneObjectType.Bin][0]

# Find the red blocks
red_blocks = [obj for obj in scene_objects if obj.get_color() == Color.Red and obj.get_object_type() == SceneObjectType.Block]

# Go to each red block, grasp it, and put it in the blue bin
for red_block in red_blocks:
    # Open the gripper
    robot.release()

    # Go to the red block
    robot.go_to_pose(red_block.get_pose())
    
    # Tilt down to the grasp position
    robot.tilt_down(0.075)
    
    # Grasp the red block
    robot.grasp()
    
    # Lift up the block
    robot.tilt_up(0.5)
    
    # Go to the blue bin
    robot.go_to_pose(blue_bin.get_pose())
    
    # Place the block down
    robot.tilt_down(0.5)
    
    # Open the gripper
    robot.release()
    
###
    
"""

In [4]:
prompt="""
# Lift up the yellow block and move it left by 30 degrees.

"""

### Submit the request to OpenAI

In [8]:
response = openai.Completion.create(engine=openai_engine, prompt=template+good_examples+prompt, max_tokens=300, temperature=0.5, stop="###")
response_text = response["choices"][0]["text"]
print(prompt+response_text)


# Lift up the yellow block and move it left by 30 degrees.


# Initialize the robot
robot = RobotController()

# Get the list of objects in the scene
scene_objects = robot.get_scene_objects()

# Find the yellow block
yellow_block = [obj for obj in scene_objects if obj.get_color() == Color.Yellow and obj.get_object_type() == SceneObjectType.Block][0]

# Open the gripper
robot.release()

# Go to the yellow block
robot.go_to_pose(yellow_block.get_pose())

# Tilt down to the grasp position
robot.tilt_down(0.075)

# Grasp the yellow block
robot.grasp()

# Lift up the block
robot.tilt_up(0.5)

# Move left 30 degrees
robot.rotate_left(math.pi / 6)

# Place the block down
robot.tilt_down(0.5)

# Drop the yellow block
robot.release()




### Run the program
First, load up the full program and make sure it looks good.

In [9]:
f = open("robot_implementation.py","r")
implementation = f.read()

full_code = implementation+prompt+response_text
print(full_code)

# Python 3
# Interface for a robot arm

import enum
import roslibpy
import time
import math
import os
from typing import List

class Color(enum.Enum):
    Blue =   "blue"   #blue
    Green =  "green"  #green
    Yellow = "yellow" #yellow
    Red =    "red"    #red
    Orange = "orange" #orange
    Black =  "black"  #black
    
class SceneObjectType(enum.Enum):
    Block =  "block"  #block
    Bin =    "bin"    #bin
    Ball =   "ball"   #ball
    
class Pose:
    def __init__(self, serialized):
        self.position = serialized["position"]
        self.orientation = serialized["orientation"]
        self.serialized = serialized
    
    def as_dict(self):
        return self.serialized;

class SceneObject:
    def __init__(self, serialized):
        self.color = Color(serialized["color"])
        self.object_type = SceneObjectType(serialized["object_type"])
        self.pose = Pose(serialized["pose"])
        
    def get_color(self) -> Color:
        return self.color
    
    def ge

Time to run the program!

In [10]:
full_code = implementation+prompt+response_text
exec(full_code)

Is ROS connected? True
Calling service...
Sending release message...
Sending move_to_pose message...
Sending tilt_down message...
Sending grasp message...
Sending tilt_up message...
Sending rotate_left message...
Sending tilt_down message...
Sending release message...


# Appendix
### Initialize ROS environment

In [ ]:
!source /opt/ros/noetic/setup.bash
!source ~/catkin_ws/devel/local_setup.sh

Run this in another command window:

    `
    source /opt/ros/noetic/setup.bash
    source ~/catkin_ws/devel/setup.sh
    source ~/catkin_ws/devel/local_setup.sh
    roslaunch niryo_moveit part_3.launch
    `

In [ ]:
! rostopic list

In [ ]:
! rostopic pub /unity/pickup_block std_msgs/String HelloUnity

In [ ]:
import roslibpy
import time

client = roslibpy.Ros(host='localhost', port=8080)
client.run()
print('Is ROS connected?', client.is_connected)

#service = roslibpy.Service(client, '/rosout/get_loggers', 'roscpp/GetLoggers')
#request = roslibpy.ServiceRequest()


#print('Calling service...')
#result = service.call(request)
#print('Service response: {}'.format(result['loggers']))

#talker = roslibpy.Topic(client, '/unity/pickup_block', 'std_msgs/String')

#talker.publish(roslibpy.Message({'data': 'yellow_block'}))
#print('Sending message...')
#time.sleep(1)

#talker = roslibpy.Topic(client, '/unity/pick_and_place', 'niryo_moveit/PickAndPlace')


#talker.unadvertise()
#client.terminate()


In [ ]:
talker.publish(roslibpy.Message({'target': 'blue_block', 'targetPlacement': 'red_bin'}))
print('Sending message...')
time.sleep(1)

In [ ]:
print('Calling service...')
result = service.call(request)
print('Service response: {}'.format(result['loggers']))

In [ ]:
# Python 3
# Interface for a robot arm

import enum
import roslibpy
import time
import os

class Color(enum.Enum):
    Blue =   "blue"   #blue
    Green =  "green"  #green
    Yellow = "yellow" #yellow
    Red =    "red"    #red
    Orange = "orange" #orange
    Black =  "black"  #black
    
class SceneObjectType(enum.Enum):
    Block =  "block"  #block
    Bin =    "bin"    #bin
    Ball =   "ball"   #ball
    
class Pose:
    position: str
    orientation: str

    def __init__(self, json):
        self.position = json["position"]
        self.orientation = json["orientation"]

class SceneObject:
    color: Color
    object_type: SceneObjectType
    pose: Pose
        
    def __init__(self, json):
        self.color = Color(json["color"])
        self.object_type = SceneObjectType(json["object_type"])
        self.pose = Pose(json["pose"])

class RobotController:
    def __init__(self):
        self.ros_client = roslibpy.Ros(host='localhost', port=8080)
        self.ros_client.run()
        print('Is ROS connected?', self.ros_client.is_connected)
        self.talker = roslibpy.Topic(self.ros_client, '/unity/pick_and_place', 'niryo_moveit/PickAndPlace')
        self.service = roslibpy.Service(self.ros_client, '/unity/object_pose_svc', 'niryo_moveit/ObjectPoseService')
        self.get_scene_objects_svc = roslibpy.Service(self.ros_client,
                                                      '/unity/get_scene_objects_svc',
                                                      'niryo_moveit/GetSceneObjectsService')
    
    def get_scene_objects(self):
        """Gets all objects in the visual scene."""
        request = roslibpy.ServiceRequest()
        
        print('Calling service...')
        result = self.get_scene_objects_svc.call(request)
        found_objects = []
        for scene_obj_json in result["scene_objects"]:
            found_objects.append(SceneObject(scene_obj_json))
        
        return found_objects
    
        
    def get_object_pose(self, name):
        """
        Finds the pose of an object matching the specified name.
        """
        request = roslibpy.ServiceRequest()
        request["object_name"] = name

        print('Calling service...')
        result = self.service.call(request)
        
        return result["object_pose"]
    
    def pick_and_place(self, pick_pose, place_pose):
        """The robot will pick up the object at target_pose and move it to the placement_pose"""
        self.talker.publish(roslibpy.Message({'pick_pose': pick_pose, 'place_pose': place_pose}))
        print('Sending message...')
        time.sleep(100)
        
    def __del__(self):
        print('Destructor called. Shutting down ROS connection.')
        #self.talker.unadvertise()
        self.ros_client.terminate()
    
###
        
# Pick up the blue block and put in in the red bin.
# The robot can only grasp one object at a time.
def main():
    robot = RobotController()
    
    scene_objects = robot.get_scene_objects()
    
    # Find blue block and red bin
    blue_block = None
    red_bin = None
    for obj in scene_objects:
        if obj.color == Color.Blue and obj.object_type == SceneObjectType.Block:
            blue_block = obj
        elif obj.color == Color.Red and obj.object_type == SceneObjectType.Bin:
            red_bin = obj
            
    print (str(blue_block))
    print (str(red_bin))
    
    #pick_pose = robot.get_object_pose("blue_block")
    #print ("blue_block pose: ", pick_pose)
    
    #place_pose = robot.get_object_pose("black_bin")
    #print ("black_bin pose: ", place_pose)
    
    #robot.pick_and_place(pick_pose, place_pose)
    
main()
    
###